In [1]:
# pip install efficient-apriori

In [2]:
import pandas as pd
import json
import numpy as np
import collections
from efficient_apriori import apriori
pd.options.mode.chained_assignment = None

In [3]:
credits = pd.read_csv(r'credits.csv')
movies1 = pd.read_csv(r'movies_metadata.csv') #,'original_title','production_companies','revenue','budget'

In [4]:
movies1

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,17-11-2011,0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3
45460,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,01-08-2003,0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6
45461,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,21-10-1917,0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0


In [5]:
#converting genres column from dictionay to list type.

mv = movies1.copy()
import ast
def dict_to_list(mv,col):
    temp = []
    for i in range(len(mv[col])):
        l =  ast.literal_eval(mv[col][i])
        mv[col][i] = [d["name"] for d in l]
#         print(mv[col][i])
        temp.append(tuple([d["name"] for d in l]))
    return temp
    
genres = dict_to_list(mv,"genres")
# genres

In [6]:
# Support (A ⇒ B): s = P(A, B) start with 0.05
# Confidence (A ⇒ B): c = P(B | A) start with 0.8
# Lift (A ⇒ B): L = c/P(B)

min_support = 0.015 #len(genres)*
min_confidence = 0.05
print(min_support)
itemsets, rules = apriori(genres,min_support,min_confidence=min_confidence)

0.015


In [7]:
rules

[{Adventure} -> {Action},
 {Action} -> {Adventure},
 {Comedy} -> {Action},
 {Action} -> {Comedy},
 {Crime} -> {Action},
 {Action} -> {Crime},
 {Drama} -> {Action},
 {Action} -> {Drama},
 {Science Fiction} -> {Action},
 {Action} -> {Science Fiction},
 {Thriller} -> {Action},
 {Action} -> {Thriller},
 {Comedy} -> {Adventure},
 {Adventure} -> {Comedy},
 {Drama} -> {Adventure},
 {Adventure} -> {Drama},
 {Family} -> {Adventure},
 {Adventure} -> {Family},
 {Fantasy} -> {Adventure},
 {Adventure} -> {Fantasy},
 {Family} -> {Animation},
 {Animation} -> {Family},
 {Crime} -> {Comedy},
 {Comedy} -> {Crime},
 {Drama} -> {Comedy},
 {Comedy} -> {Drama},
 {Family} -> {Comedy},
 {Comedy} -> {Family},
 {Fantasy} -> {Comedy},
 {Comedy} -> {Fantasy},
 {Romance} -> {Comedy},
 {Comedy} -> {Romance},
 {Drama} -> {Crime},
 {Crime} -> {Drama},
 {Mystery} -> {Crime},
 {Crime} -> {Mystery},
 {Thriller} -> {Crime},
 {Crime} -> {Thriller},
 {Family} -> {Drama},
 {Fantasy} -> {Drama},
 {Foreign} -> {Drama},
 {Hist

In [8]:
lhs = [list(r.lhs) for r in rules]
rhs = [list(r.rhs) for r in rules]

In [9]:
# lhs,rhs

In [10]:
mv2 = mv.copy()
for i in range(len(mv2['genres'])):
#     print(i)
    if mv2['genres'][i] not in lhs:
        mv2.drop([i],inplace=True)
        movies1.drop([i],inplace=True)
len(mv2)

16526

In [11]:
mv2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
6,False,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,15-12-1995,0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141
16,False,NaN,16500000,"[Drama, Romance]",NaN,4584,tt0114388,en,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",...,13-12-1995,135000000,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,7.2,364
21,False,NaN,0,"[Drama, Thriller]",NaN,1710,tt0112722,en,Copycat,An agoraphobic psychologist and a female detec...,...,27-10-1995,0,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One man is copying the most notorious killers ...,Copycat,False,6.5,199
24,False,NaN,3600000,"[Drama, Romance]",http://www.mgm.com/title_title.do?title_star=L...,451,tt0113627,en,Leaving Las Vegas,"Ben Sanderson, an alcoholic Hollywood screenwr...",...,27-10-1995,49800000,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,I Love You... The Way You Are.,Leaving Las Vegas,False,7.1,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45441,False,NaN,0,[Fantasy],NaN,44333,tt0135179,fr,Les cartes vivantes,A bearded magician holds up a large playing ca...,...,01-01-1905,0,3.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,The Living Playing Cards,False,6.1,8
45451,False,NaN,0,"[Comedy, Drama]",NaN,420346,tt4130180,en,The Morning After,The Morning After is a feature film that consi...,...,11-01-2015,0,79.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,What happened last night?,The Morning After,False,4.0,2
45455,False,NaN,0,[Horror],NaN,289923,tt0252966,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,...,03-10-2000,0,30.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Do you know what happened 50 years before ""The...",The Burkittsville 7,False,7.0,1
45456,False,NaN,0,[Science Fiction],NaN,222848,tt0112613,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,...,01-01-1995,0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Caged Heat 3000,False,3.5,1


In [12]:
mv2.to_csv('mov2.csv',index=False)

In [13]:
# cols = ['adult','id','original_language','popularity','release_date','runtime','vote_average']
# movies = movies1[cols]
# movies

In [14]:
# movies['adult']= movies['adult'].astype(int)

# # movies

In [15]:
# #removing nan values.
# runtime = movies.runtime.mean()
# movies['runtime'] = movies['runtime'].fillna(value=runtime)

# lang = "notKnown"
# movies['original_language'] = movies['original_language'].fillna(value=lang)

# date = movies.release_date.mode().get(key=0)
# print(movies)
# movies['release_date'] = movies['release_date'].fillna(value=date)

In [16]:
# #converting date column from object dtype to int.
# movies['release_date'] = movies['release_date'].str.replace("-","").astype(int)
# movies['release_date'] = movies['release_date']%10000

In [17]:
# #replacing original language with its frequency.

# lang_fq = movies['original_language'].value_counts()
# lang_freq = lang_fq.to_dict()
# lang = movies['original_language'].unique()

# for i in lang:
#     movies.loc[ movies['original_language'] == i, 'original_language'] = lang_freq[i]  
# # movies

In [18]:
# for i in range(len(credits['cast'])):
#     l =  ast.literal_eval(credits['cast'][i])
#     if(l!=[]):
#         credits['cast'][i] = l[0]['name']
#     else:
#         credits['cast'][i] = "notKnown"

# for i in range(len(credits['crew'])):
#     l =  ast.literal_eval(credits['crew'][i])
#     if(l!=[]):
#         for idx in l:
#             if(idx['job']=="Director"):
#                 credits['crew'][i] = idx['name']
#     else:
#         credits['crew'][i] = "notKnown"

In [19]:
# #giving unique index for cast and crew.
# def index_alot(df,col):
#     idx = 0
#     dict_idx = {}
#     for i in df[col].unique():
# #         print(i)
#         dict_idx[i] = idx
#         idx+=1;
# #         print(idx)
#     return dict_idx

# #replacing cast and crew names with the alloted idx.
# def index_replace(df,col,idx):
#     for i in idx:
#         df.loc[df[col] == i, col] = idx[i] 

In [20]:
# crewid = index_alot(credits,'crew')
# castid = index_alot(credits,'cast')

# index_replace(credits,'crew',crewid)
# index_replace(credits,'cast',castid)

# df = pd.merge(movies,credits,on="id",how='inner',sort=False)
# df

In [21]:
# cols = ['cast','crew','original_language','popularity','release_date','runtime','vote_average']
# scaled = df.copy()
  
# # scaling (normalizing)
# for c in cols:
#     scaled[c].fillna(scaled[c].mean(), inplace=True)
#     scaled[c] = scaled[c]  / scaled[c].abs().max()
    
# for c in cols:
#     movies[c] = scaled[c]
# movies

In [22]:
# # df.insert(0,"id",df['id'],True)
# movies.to_csv('mov.csv',index=False)


In [23]:
# lhs
with open(r'lhs.txt', 'w') as fp:
    fp.write(''.join(str(lhs)))

In [24]:
with open(r'rhs.txt', 'w') as fp:
    fp.write(''.join(str(rhs)))

In [25]:

# with open(r'lhs.txt', 'w') as fp:
#     fp.write(''.join(str(lhs)))
    
# with open(r'rhs.txt', 'w') as fp:
#     fp.write('\n'.join(str(rhs)))

In [26]:
# cos_dist = []

# for i in range(len(mov1[cols])):
    
#     list2 = feat2.loc[i].tolist()
#     result = 1 - spatial.distance.cosine(fin, list2)
#     cos_dist.append([result,i])

In [27]:
# c = 0
# for i in res:
#     for j in range(len(mov1['id'])):
#         if (i == j):
#             c+=1
#         elif(j > i):
#             break
# print(c)